<a href="https://colab.research.google.com/github/RoboMaroof/LLM-Applications-Building-Blocks/blob/main/03_Quantization/02_GPTQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resources

https://towardsdatascience.com/the-ultimate-handbook-for-llm-quantization-88bb7cb0d9d7#d2c7

# Installs and Imports

In [ ]:
!pip install auto_gptq
!pip install optimum
!pip install -U accelerate bitsandbytes datasets peft transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    GPTQConfig,
)
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import torch
import gc
from google.colab import userdata

# Comparison functions

## Model size

In [ ]:
def get_model_size(model):
    return sum(p.numel() * p.element_size() for p in model.parameters()) / 1024**2

## Inference test

In [ ]:
def inference_test(model):
    # Performance comparison (inference speed)
    input_text = "Once upon a time"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

    print("\nInference speed test:")
    with torch.no_grad():
        # Warm-up run
        _ = model.generate(input_ids, max_new_tokens=50)

        # Timed run
        start_time = torch.cuda.Event(enable_timing=True)
        end_time = torch.cuda.Event(enable_timing=True)

        start_time.record()
        output = model.generate(input_ids, max_new_tokens=50)
        end_time.record()

        torch.cuda.synchronize()
        print(f"Generation time: {start_time.elapsed_time(end_time):.2f} ms")

    print("\nGenerated text:")
    print(tokenizer.decode(output[0], skip_special_tokens=True))

# Model

In [ ]:
model_name = "google/gemma-2b-it"

## Load Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=userdata.get('HF_TOKEN')
)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# Full precision model

## Load Model

In [ ]:
full_precision_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    token=userdata.get('HF_TOKEN')
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Model size

In [ ]:
print(f"Quantized model size: {get_model_size(full_precision_model):.2f} MB")

Quantized model size: 9560.29 MB


## Inference speed

In [ ]:
inference_test(full_precision_model)


Inference speed test:
Generation time: 38333.68 ms

Generated text:
Once upon a time, in a quaint village nestled amidst rolling hills, lived a young woman named Elara. With eyes as bright as the morning sun and a smile that could melt the iciest of winter days, Elara possessed a heart that was as warm as the


# Clear Memory

In [ ]:
del full_precision_model
gc.collect()
torch.cuda.empty_cache()

# GPTQ Model

## Quantization config

In [ ]:
quant_config = GPTQConfig(
    bits=4,
    dataset = "c4",
    tokenizer=tokenizer
)

## Load Model

In [ ]:
model_GPTQ_int4 = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    token=userdata.get('HF_TOKEN')
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Quantizing model.layers blocks :   0%|          | 0/18 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4689: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


## Model size

In [ ]:
print(f"Quantized model size: {get_model_size(model_GPTQ_int4):.2f} MB")

Quantized model size: 1000.14 MB


## Inference speed

In [ ]:
inference_test(model_GPTQ_int4)


Inference speed test:
Generation time: 12661.32 ms

Generated text:
Once upon a time, in a quaint village nestled amidst rolling hills, there lived a young woman named Elara. With eyes as bright as the morning dew and hair as dark as the night sky, Elara possessed a heart of gold and a spirit that soared beyond the


# Push quantized model to HuggingFace

In [ ]:
model_GPTQ_int4.push_to_hub("Maroof-Mohammed/gemma-2b-GPTQ", check_pr=True, token=userdata.get('HF_TOKEN_WRITE'))
tokenizer.push_to_hub("Maroof-Mohammed/gemma-2b-GPTQ", check_pr=True, token=userdata.get('HF_TOKEN_WRITE'))

model.safetensors:   0%|          | 0.00/2.08G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Maroof-Mohammed/gemma-2b-GPTQ/commit/58ed817eb6a2dc7ccbf8301921dd7ce11b2fbe32', commit_message='Upload tokenizer', commit_description='', oid='58ed817eb6a2dc7ccbf8301921dd7ce11b2fbe32', pr_url=None, pr_revision=None, pr_num=None)

# Save quantized model locally

In [ ]:
# Save the quantized model
model_GPTQ_int4.save_pretrained("./model_GPTQ_int4", safe_serialization=True)
tokenizer.save_pretrained("./model_GPTQ_int4")

('./model_GPTQ_int4/tokenizer_config.json',
 './model_GPTQ_int4/special_tokens_map.json',
 './model_GPTQ_int4/tokenizer.model',
 './model_GPTQ_int4/added_tokens.json',
 './model_GPTQ_int4/tokenizer.json')